In [1]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold
iris = datasets.load_iris() #데이터를 로드
features = iris.data # 특성과 타깃을 만듭니다
target = iris.target
thresholder = VarianceThreshold(threshold=.5) # 기준값을 만듭니다.
features_high_variance = thresholder.fit_transform(features) # 기준값보다 높은 특성을 선택합니다.
features_high_variance[0:3] # 선택한 특성을 확인
thresholder.variances_ # 분산을 확인합니다.

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

In [2]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() # 특성 행렬을 표준화합니다.
features_std = scaler.fit_transform(features)
selector = VarianceThreshold() # 각 특성의 분산을 계산합니다.
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

In [3]:
from sklearn.feature_selection import VarianceThreshold
features = [[0, 1, 0], # 예제 특성 행렬
[0, 1, 1], # 특성 0: 80%가 클래스 0
[0, 1, 0], # 특성 1: 80%가 클래스 1
[0, 1, 1], # 특성 2: 60%가 클래스 0, 40%는 클래스 1
[1, 0, 0]]
# 분산을 기준으로 선택합니다.
thresholder = VarianceThreshold(threshold=(.75 * (1 - .75)))
thresholder.fit_transform(features)
thresholder.variances_
import numpy as np
np.var(features, axis=0) #넘파이 var 함수를 사용하여 분산을 계산합니다

array([0.16, 0.16, 0.24])

In [4]:
import pandas as pd
import numpy as np
# 상관관계가 큰 두 개의 특성을 가진 특성 행렬을 만듭니다.
features = np.array([[1, 1, 1], [2, 2, 0], [3, 3, 1], [4, 4, 0], [5, 5, 1],
[6, 6, 0], [7, 7, 1], [8, 7, 0], [9, 7, 1]])
dataframe = pd.DataFrame(features) # 특성 행렬을 DataFrame으로 변환
corr_matrix = dataframe.corr().abs() # 상관관계 행렬을 만듭니다.
# 상관관계 행렬의 상삼각(upper triangle) 행렬을 선택합니다.
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
# 상관 계수가 0.95보다 큰 특성 열의 인덱스를 찾습니다.
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

In [6]:
dataframe.drop(dataframe.columns[to_drop], axis=1).head(3) # 특성을 삭제합니다.

,0,2
0,1,1
1,2,0
2,3,1


In [7]:
dataframe.corr() #상관관계 행렬
upper #상관관계 행렬의 상삼각 행렬

,0,1,2
0,NaN,0.976103,0.000000
1,NaN,NaN,0.034503
2,NaN,NaN,NaN


In [8]:
np.corrcoef(features, rowvar=False)

array([[ 1.        ,  0.97610336,  0.        ],
       [ 0.97610336,  1.        , -0.03450328],
       [ 0.        , -0.03450328,  1.        ]])

In [9]:
np.triu(np.ones((4, 4)), k=2)
np.tril(np.ones((4, 4)), k=0)

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

### 분류 작업에 관련 없는 특성 삭제
- 범주형 타깃 벡터에서 관련 없는 특성을 삭제하기 위해 타깃 벡터 사이의 카이제곱 통계를 계산합니다.

- 카이제곱 통계는 두 범주형 벡터의 독립성을 평가합니다.

- 카이제곱 통계는 범주형 특성의 각 클래스별 샘플 빈도와 이 특성이 타깃 벡터와 독립적이라면 기대할 수 있는 값
  사이의 차이입니다

- 카이제곱 특성은 관찰 빈도와 전혀 관계가 없다고 기대하는 빈도 사이에 얼마나 큰 차이가 있는지 알려주는 하나
   의 숫자입니다.
-  특성과 타깃 벡터 사이의 카이제곱 통계를 계산하면 둘 사이의 독립성을 측정할 수 있습니다.

In [10]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
iris = load_iris() # 데이터 로드
features = iris.data
target = iris.target
features = features.astype(int) # 범주형 데이터를 정수형으로 변환
chi2_selector = SelectKBest(chi2, k=2) # 카이제곱 통계값이 가장 큰 특성 두 개를 선택
features_kbest = chi2_selector.fit_transform(features, target)
print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2


In [11]:
# F-값이 가장 높은 특성 두 개를 선택합니다.
fvalue_selector = SelectKBest(f_classif, k=2)
features_kbest = fvalue_selector.fit_transform(features, target)
print("원본 특성 개수:", features.shape[1]) # 결과 확인
print("줄어든 특성 개수:", features_kbest.shape[1])
# 특정 특성 개수를 선택하는 대신 Selectpercentile를 사용하여 특성의 상위 n 퍼센트를 선택할 수 있습니다.
from sklearn.feature_selection import SelectPercentile
# 가장 큰 F-값의 상위 75% 특성을 선택합니다.
fvalue_selector = SelectPercentile(f_classif, percentile=75)
features_kbest = fvalue_selector.fit_transform(features, target)
print("원본 특성 개수:", features.shape[1]) # 결과 선택
print("줄어든 특성 개수:", features_kbest.shape[1])

원본 특성 개수: 4
줄어든 특성 개수: 2
원본 특성 개수: 4
줄어든 특성 개수: 3


In [14]:
target
#특성 행렬의 차원을 (3, 50, 4)로 바꾸어 클래스별 합을 구합니다.
observed = np.sum(features.reshape(3, 50, 4), axis=1)
observed
#특성 타깃과 전혀 관계없다면 기대 빈도는 전체 합을 클래스 개수 3으로 나눈 값이 됩니다.
expected = features.sum(axis=0) / 3
expected
#카이제곱 공식에 위에서 구한 observed와 expected를 대입합니다.
np.sum((observed - expected)**2 / expected, axis=0)
#카이제곱 값이 큰 세 번째, 네 번째 특성이 선택됩니다. chi2_selector객체의 scores_속성에 저장
chi2_selector.scores_

array([ 10.28712871,   5.02267003, 133.06854839,  74.27906977])

In [16]:
##전체 평균과 클래스 평균을 계산
total_mean = np.mean(features, axis=0)
total_mean
class_mean = np.mean(features.reshape(3, 50, 4), axis=1)
class_mean

array([[4.6 , 3.04, 1.  , 0.  ],
       [5.48, 2.32, 3.82, 1.  ],
       [6.08, 2.58, 5.1 , 1.58]])

In [17]:
#ss_total 계산
ss_between = np.sum(50 * (class_mean - total_mean)**2, axis=0)
ss_between
ss_total = np.sum((features - total_mean)**2, axis=0)
ss_total

array([105.57333333,  42.27333333, 467.89333333,  76.06      ])

In [18]:
#ss_beteen과 ss_tatal을 F-값 공식에 대입
f = (ss_between/(3-1)) / ((ss_total-ss_between)/(150-3))
f
fvalue_selector.scores_ #F-값 scores_속성에서 확인

array([  81.19715 ,   33.715004, 1160.0116  ,  385.483   ], dtype=float32)

In [20]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import RFECV
from sklearn import datasets, linear_model
# 특성 행렬과 타깃 벡터를 생성합니다.
features, target = make_regression(n_samples = 10000,
                                n_features = 100,
                                n_informative = 2,
                                random_state = 1)
# 선형 회귀 모델을 만듭니다.
ols = linear_model.LinearRegression()


In [21]:
# 재귀적으로 특성을 제거합니다.
rfecv = RFECV(estimator=ols, step=1, scoring="neg_mean_squared_error")
rfecv.fit(features, target)
rfecv.transform(features)

C:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


array([[ 0.00850799,  0.7031277 ],
       [-1.07500204,  2.56148527],
       [ 1.37940721, -1.77039484],
       ...,
       [-0.80331656, -1.60648007],
       [ 0.39508844, -1.34564911],
       [-0.55383035,  0.82880112]])

In [24]:
rfecv.n_features_ # 최선의 특성 개수
rfecv.support_ # 선택된 특성이 표시된 불리언 마스크
rfecv.ranking_ # 특성의 순위: 최고(1)에서 최악(96)까지


array([82, 84, 74, 33, 81,  1, 18, 46, 57, 67, 45,  7, 58, 52, 78,  8,  5,
       73, 31, 11, 43, 14, 34, 83, 21, 96, 20, 41, 94, 90, 71, 47, 30, 27,
       89, 50, 25, 69, 86,  1, 76, 19, 97, 88,  9, 16, 23, 80, 75, 54, 91,
       12, 65, 59, 24, 32,  4, 26, 10, 42, 72,  2, 87, 40, 66,  3, 92, 17,
       39, 35, 13, 79, 38,  6, 53, 60, 22, 61, 28, 95, 93, 36, 99, 48, 51,
       68, 37, 70, 15, 98, 56, 29, 44, 63, 49, 64, 77, 85, 55, 62])

In [26]:
from sklearn.feature_selection import RFE
rfe = RFE(estimator=ols, n_features_to_select=3)
rfe.fit(features, target)
rfe.transform(features)


array([[ 0.00850799,  0.7031277 , -1.16432076],
       [-1.07500204,  2.56148527, -1.3936433 ],
       [ 1.37940721, -1.77039484,  1.02058188],
       ...,
       [-0.80331656, -1.60648007, -0.64067478],
       [ 0.39508844, -1.34564911, -1.91468325],
       [-0.55383035,  0.82880112,  0.27936745]])

In [27]:
np.all(rfe.support_ == rfecv.support_)

False